# DeepLabCut Toolbox
https://github.com/AlexEMG/DeepLabCut

Nath\*, Mathis\* et al. *Using DeepLabCut for markerless pose estimation during behavior across species*, (under revision).

This notebook demonstrates the necessary steps to use DeepLabCut for your own project.
This shows the most simple code to do so, but many of the functions have additional features, so please check out the overview & the protocol paper!

This notebook illustrates how to:
- create a project
- extract training frames
- label the frames
- plot the labeled images
- create a training set
- train a network
- evaluate a network
- analyze a novel video
- create an automatically labeled video 
- plot the trajectories

*Note*: Refine a network based after the network was trained on just a few labeled images is illustrated in "Demo-labeledexample-MouseReaching.ipynb". This demo also contains an already labeled data set and is perhaps the best starting point for brand new users.

## Create a new project

It is always good idea to keep the projects seperate. This function creates a new project with subdirectories and a basic configuration file in the user defined directory otherwise the project is created in the current working directory.

You can always add new videos to the project at any stage of the project. 

In [1]:
import deeplabcut


In [29]:
import os
import subprocess

def convert_video(video_input, video_output):
    cmds = ['ffmpeg', '-i', video_input, video_output]
    subprocess.Popen(cmds)
    
Mainfolder = 'D:\\rat movies SC\\ar20\\02_15_19';
video_names = [f for f in os.listdir(Mainfolder) if f.endswith('.avi')]
video_name2s = [f.replace('.avi', '.mp4') for f in text_files ]
convert_video(os.path.join(Mainfolder,video_names[1]),os.path.join(Mainfolder,video_name2s[1]))

     
print(video_names)
print(video_name2s)

for i, item in enumerate(video_names):
 print(os.path.join(Mainfolder,video_name2s[i]))
 convert_video(os.path.join(Mainfolder,video_names[i]),os.path.join(Mainfolder,video_name2s[i]))


['12-34-58.avi', '12-35-11.avi', '12-35-24.avi', '12-35-37.avi', '12-35-59.avi', '12-36-00.avi', '12-37-29.avi', '12-37-33.avi', '12-37-35.avi', '12-37-37.avi', '12-37-40.avi', '12-41-23.avi', '12-41-26.avi', '12-41-39.avi', '12-41-56.avi', '12-41-58.avi', '12-42-01.avi', '12-42-14.avi', '12-42-27.avi', '12-42-40.avi']
['12-34-58.mp4', '12-35-11.mp4', '12-35-24.mp4', '12-35-37.mp4', '12-35-59.mp4', '12-36-00.mp4', '12-37-29.mp4', '12-37-33.mp4', '12-37-35.mp4', '12-37-37.mp4', '12-37-40.mp4', '12-41-23.mp4', '12-41-26.mp4', '12-41-39.mp4', '12-41-56.mp4', '12-41-58.mp4', '12-42-01.mp4', '12-42-14.mp4', '12-42-27.mp4', '12-42-40.mp4']
D:\rat movies SC\ar20\02_15_19\12-34-58.mp4
D:\rat movies SC\ar20\02_15_19\12-35-11.mp4
D:\rat movies SC\ar20\02_15_19\12-35-24.mp4
D:\rat movies SC\ar20\02_15_19\12-35-37.mp4
D:\rat movies SC\ar20\02_15_19\12-35-59.mp4
D:\rat movies SC\ar20\02_15_19\12-36-00.mp4
D:\rat movies SC\ar20\02_15_19\12-37-29.mp4
D:\rat movies SC\ar20\02_15_19\12-37-33.mp4
D:\rat

In [2]:
task='sc_ar17record_02272019' # Enter the name of your experiment Task
experimenter='Fassihi' # Enter the name of the experimenter
video=['D:\\Notebook\\sc\\sc-Fassihi-2018-12-16\\videos\\video14-23-08.avi'] 
video=['D:\\rat movies SC\\ar17\\01_11_19\\video12-29-40.avi'] 
video=['D:\\rat movies SC\\ar17record\\02_27_19\\18-35-49video.MP4']
#D:\rat movies SC\ar15\01_11_19
# Enter the paths of your videos you want to grab frames from.
# deeplabcut.create_new_project(task,experimenter,video, working_directory='Full path of the working directory',copy_videos=True/False) #change the working directory to where you want the folders created.
deeplabcut.create_new_project(task,experimenter,video, working_directory='D:\\rat movies SC\\ar17record\\02_27_19',copy_videos=True) #change the working directory to where you want the folders created.

Created "D:\rat movies SC\ar17record\02_27_19\sc_ar17record_02272019-Fassihi-2019-03-05\videos"
Created "D:\rat movies SC\ar17record\02_27_19\sc_ar17record_02272019-Fassihi-2019-03-05\labeled-data"
Created "D:\rat movies SC\ar17record\02_27_19\sc_ar17record_02272019-Fassihi-2019-03-05\training-datasets"
Created "D:\rat movies SC\ar17record\02_27_19\sc_ar17record_02272019-Fassihi-2019-03-05\dlc-models"
Copying the videos
D:\rat movies SC\ar17record\02_27_19\18-35-49video.MP4
Generated "D:\rat movies SC\ar17record\02_27_19\sc_ar17record_02272019-Fassihi-2019-03-05\config.yaml"

A new project with name sc_ar17record_02272019-Fassihi-2019-03-05 is created at D:\rat movies SC\ar17record\02_27_19 and a configurable file (config.yaml) is stored there. Change the parameters in this file to adapt to your project's needs.
 Once you have changed the configuration file, use the function 'extract_frames' to select frames for labeling.
. [OPTIONAL] Use the function 'add_new_videos' to add new videos

'D:\\rat movies SC\\ar17record\\02_27_19\\sc_ar17record_02272019-Fassihi-2019-03-05\\config.yaml'

## Extract frames from videos 
A key point for a successful feature detector is to select diverse frames, which are typical for the behavior you study that should be labeled.

This function selects N frames either uniformly sampled from a particular video (or folder) (algo=='uniform'). Note: this might not yield diverse frames, if the behavior is sparsely distributed (consider using kmeans), and/or select frames manually etc.

Also make sure to get select data from different (behavioral) sessions and different animals if those vary substantially (to train an invariant feature detector).

Individual images should not be too big (i.e. < 850 x 850 pixel). Although this can be taken care of later as well, it is advisable to crop the frames, to remove unnecessary parts of the frame as much as possible.

Always check the output of cropping. If you are happy with the results proceed to labeling.

In [3]:
%matplotlib inline
path_config_file = 'D:\\rat movies SC\\ar17record\\02_27_19\\sc_ar17record_02272019-Fassihi-2019-03-05\\config.yaml' #Enter the path of the config file that was just created from the above step (check the folder)
deeplabcut.extract_frames(path_config_file,'automatic','uniform') #there are other ways to grab frames, such as by clustering 'kmeans'; please see the paper. 
#deeplabcut.extract_frames(path_config_file,'automatic','uniform',crop=True, checkcropping=True) #there are other ways to grab frames, such as by clustering 'kmeans'; please see the paper. 

#You can change the cropping to false, then delete the checkcropping part!

Reading config file successfully...
Extracting frames based on uniform ...
Uniformly extracting of frames from 0.0  seconds to 1200.0  seconds.

Frames are selected.
You can now label the frames using the function 'label_frames'.


## Label the extracted frames
Only videos in the config file can be used to extract the frames. Extracted labels for each video are stored in the project directory under the subdirectory **'labeled-data'**. Each subdirectory is named after the name of the video. The toolbox has a labeling toolbox which could be used for labeling. 

In [6]:
%gui wx
path_config_file = 'D:\\rat movies SC\\ar15record\\02_27_19\\sc_ar15record_02272019-Fassihi-2019-03-12\\config.yaml' #Enter the path of the config file that was just created from the above step (check the folder)

deeplabcut.label_frames(path_config_file)

ERROR:root:Cannot activate multiple GUI eventloops


Working on folder: 18-04-40video
You can now check the labels, using 'check_labels' before proceeding. Then,  you can use the function 'create_training_dataset' to create the training dataset.


**Check the labels**

Checking if the labels were created and stored correctly is beneficial for training, since labeling is one of the most critical parts for creating the training dataset. The DeepLabCut toolbox provides a function `check\_labels'  to do so. It is used as follows:

In [ ]:
print(path_config_file)

In [7]:
deeplabcut.check_labels(path_config_file) #this creates a subdirectory with the frames + your labels

Creating images with labels by Fassihi.
They are stored in the following folder: D:\rat movies SC\ar15record\02_27_19\sc_ar15record_02272019-Fassihi-2019-03-12\labeled-data\18-04-40video_labeled.
If all the labels are ok, then use the function 'create_training_dataset' to create the training dataset!


If the labels need adjusted, you can use the refinement GUI to move them around! Check that out below.

## Create a training dataset
This function generates the training data information for DeepCut (which requires a mat file) based on the pandas dataframes that hold label information. The user can set the fraction of the training set size (from all labeled image in the hd5 file) in the config.yaml file. While creating the dataset, the user can create multiple shuffles. 

After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**.

Now it is the time to start training the network!

In [9]:
deeplabcut.create_training_dataset(path_config_file)

D:\rat movies SC\ar15record\02_27_19\sc_ar15record_02272019-Fassihi-2019-03-12\training-datasets\iteration-0\UnaugmentedDataSet_sc_ar15record_02272019Mar12  already exists!
D:\rat movies SC\ar15record\02_27_19\sc_ar15record_02272019-Fassihi-2019-03-12\dlc-models\iteration-0\sc_ar15record_02272019Mar12-trainset95shuffle1  already exists!
D:\rat movies SC\ar15record\02_27_19\sc_ar15record_02272019-Fassihi-2019-03-12\dlc-models\iteration-0\sc_ar15record_02272019Mar12-trainset95shuffle1//train  already exists!
D:\rat movies SC\ar15record\02_27_19\sc_ar15record_02272019-Fassihi-2019-03-12\dlc-models\iteration-0\sc_ar15record_02272019Mar12-trainset95shuffle1//test  already exists!
The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


## Start training - If you want to use a CPU, continue. 
### If yu want to use your GPU, you need to exit here and either work from the Docker container, your own TensorFlow installation in an Anaconda env

This function trains the network for a specific shuffle of the training dataset. 

In [ ]:
#path_config_file = 'D:\\rat movies SC\\ar19\\02_07_19\\sc_ar19_02072019-Fassihi4-2019-02-13\\config.yaml' #Enter the path of the config file that was just created from the above step (check the folder)
#path_config_file = 'D:\\rat movies SC\\ar15record\\02_27_19\\sc_ar15record_02272019-Fassihi-2019-03-12\\config.yaml' #Enter the path of the config file that was just created from the above step (check the folder)
path_config_file = 'D:\\rat movies SC\\ar17record\\02_27_19\\sc_ar17record_02272019-Fassihi-2019-03-05\\config.yaml'

deeplabcut.train_network(path_config_file)

Config:
{'all_joints': [[0], [1], [2]],
 'all_joints_names': ['Nose', 'Snout', 'Head'],
 'batch_size': 1,
 'bottomheight': 400,
 'crop': True,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets\\iteration-1\\UnaugmentedDataSet_sc_ar17record_02272019Mar5\\sc_ar17record_02272019_Fassihi95shuffle1.mat',
 'dataset_type': 'default',
 'display_iters': 1000,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': 'C:\\Users\\Arash\\Anaconda3\\envs\\Myenv2\\lib\\site-packages\\deeplabcut\\pose_estimation_tensorflow\\models\\pretrained\\resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'leftwidth': 400,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 0.05,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'max_input_size': 1000,
 'mean_pixel': [123.68, 116.779, 103.939],
 'metadataset': 'training-datasets\\iteration-1\\UnaugmentedDataSet_sc_ar17record_02272019Mar5\\Documentation_data-sc_ar17record_0227

INFO:tensorflow:Restoring parameters from C:\Users\Arash\Anaconda3\envs\Myenv2\lib\site-packages\deeplabcut\pose_estimation_tensorflow\models\pretrained\resnet_v1_50.ckpt


Restoring parameters from C:\Users\Arash\Anaconda3\envs\Myenv2\lib\site-packages\deeplabcut\pose_estimation_tensorflow\models\pretrained\resnet_v1_50.ckpt


Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'weigh_only_present_joints': False, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': 'D:\\rat movies SC\\ar17record\\02_27_19\\sc_ar17record_02272019-Fassihi-2019-03-05\\dlc-models\\iteration-1\\sc_ar17record_02272019Mar5-trainset95shuffle1\\train\\snapshot', 'log_dir': 'log', 'global_scale': 0.8, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_huber_loss': True, 'optimizer': 'sgd', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, 'regularize': False, 'weight_decay': 0.0001, 'mirror': False, 'crop_pad': 0, 'scoremap_dir': 'test', 'dataset_type': 'default', 'use_gt_segm': False, 'batch_size': 1, 'video': False, 'video_batch': False, 'crop': True, 'cropratio': 0.4, 'minsize': 100, 'leftwidth': 400, 'rightwidth': 400, 'topheight': 400, 'bottomheight': 400, 'all_joints': [[0], [1], [

iteration: 1000 loss: 0.0150 lr: 0.005
iteration: 2000 loss: 0.0057 lr: 0.005
iteration: 3000 loss: 0.0044 lr: 0.005
iteration: 4000 loss: 0.0037 lr: 0.005
iteration: 5000 loss: 0.0034 lr: 0.005
iteration: 6000 loss: 0.0032 lr: 0.005
iteration: 7000 loss: 0.0029 lr: 0.005
iteration: 8000 loss: 0.0027 lr: 0.005
iteration: 9000 loss: 0.0026 lr: 0.005
iteration: 10000 loss: 0.0025 lr: 0.005
iteration: 11000 loss: 0.0043 lr: 0.02
iteration: 12000 loss: 0.0033 lr: 0.02
iteration: 13000 loss: 0.0030 lr: 0.02
iteration: 14000 loss: 0.0029 lr: 0.02
iteration: 15000 loss: 0.0027 lr: 0.02
iteration: 16000 loss: 0.0027 lr: 0.02
iteration: 17000 loss: 0.0025 lr: 0.02
iteration: 18000 loss: 0.0024 lr: 0.02


## Start evaluating
This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results**

In [4]:
#deeplabcut.evaluate_network(path_config_file)
path_config_file = 'D:\\rat movies SC\\ar17record\\02_27_19\\sc_ar17record_02272019-Fassihi-2019-03-05\\config.yaml'

deeplabcut.evaluate_network(path_config_file,Shuffles=[1], plotting=True)

D:\rat movies SC\ar17record\02_27_19\sc_ar17record_02272019-Fassihi-2019-03-05/evaluation-results/  already exists!
D:\rat movies SC\ar17record\02_27_19\sc_ar17record_02272019-Fassihi-2019-03-05\evaluation-results\iteration-1\sc_ar17record_02272019Mar5-trainset95shuffle1  already exists!
Running  DeepCut_resnet50_sc_ar17record_02272019Mar5shuffle1_80000  with # of trainingiterations: 80000
This net has already been evaluated!
The network is evaluated and the results are stored in the subdirectory 'evaluation_results'.
If it generalizes well, choose the best model for prediction and update the config file with the appropriate index for the 'snapshotindex'.
Use the function 'analyze_video' to make predictions on new videos.
Otherwise consider retraining the network (see DeepLabCut workflow Fig 2)


## Start Analyzing videos
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides. 

In [5]:
#path_config_file = 'D:\\Notebook\\sc\\sc-Fassihi-2018-12-15\\config.yaml' # Enter the path of the config file that was just created from the above step (check the folder)

#videofile_path = ['D:\\Notebook\\sc\\sc-Fassihi-2018-12-16\\videos\\video14-23-50.avi'] #Enter the list of videos to analyze.
#path_config_file = 'D:\\rat movies SC\\ar19\\02_07_19\\sc_ar19_02072019-Fassihi4-2019-02-13\\config.yaml' #Enter the path of the config file that was just created from the above step (check the folder)
#path_config_file ='D:\\rat movies SC\\ar17record\\02_27_19\\sc_ar17record_02272019-Fassihi-2019-03-05\\config.yaml' #Enter the path of the config file that was just created from the above step (check the folder)

import os
Mainfolder = 'D:\\rat movies SC\\ar15record\\02_27_19'
Mainfolder = 'D:\\rat movies SC\\ar17record\\02_27_19\\analyzed'

f = os.listdir(Mainfolder)
text_files = [f for f in os.listdir(Mainfolder) if f.endswith('.mp4')]

my_list = text_files
thisApen = Mainfolder+'kir'
thisApen = thisApen.replace("kir", "\\")
string = thisApen
my_new_list = [ string + x for x in my_list]
print (my_new_list)
#my_new_list = ['D:\\Notebook\\sc\\sc-Fassihi-2018-12-16\\videos\\video14-28-31.avi', 'D:\\Notebook\\sc\\sc-Fassihi-2018-12-16\\videos\\video14-28-41.avi', 'D:\\Notebook\\sc\\sc-Fassihi-2018-12-16\\videos\\video14-28-52.avi']

#deeplabcut.analyze_videos(path_config_file,videofile_path)
# my_new_list was created in prevouse cell as all videos in the folder 
deeplabcut.analyze_videos(path_config_file,my_new_list,shuffle=1, save_as_csv=True)

['D:\\rat movies SC\\ar17record\\02_27_19\\analyzed\\18-27-47video.mp4', 'D:\\rat movies SC\\ar17record\\02_27_19\\analyzed\\18-31-00video.mp4', 'D:\\rat movies SC\\ar17record\\02_27_19\\analyzed\\18-31-30video.mp4', 'D:\\rat movies SC\\ar17record\\02_27_19\\analyzed\\18-31-32video.mp4', 'D:\\rat movies SC\\ar17record\\02_27_19\\analyzed\\18-31-59video.mp4', 'D:\\rat movies SC\\ar17record\\02_27_19\\analyzed\\18-32-12video.mp4', 'D:\\rat movies SC\\ar17record\\02_27_19\\analyzed\\18-32-47video.mp4', 'D:\\rat movies SC\\ar17record\\02_27_19\\analyzed\\18-33-04video.mp4', 'D:\\rat movies SC\\ar17record\\02_27_19\\analyzed\\18-33-21video.mp4', 'D:\\rat movies SC\\ar17record\\02_27_19\\analyzed\\18-33-41video.mp4', 'D:\\rat movies SC\\ar17record\\02_27_19\\analyzed\\18-34-00video.mp4', 'D:\\rat movies SC\\ar17record\\02_27_19\\analyzed\\18-34-17video.mp4', 'D:\\rat movies SC\\ar17record\\02_27_19\\analyzed\\18-34-37video.mp4', 'D:\\rat movies SC\\ar17record\\02_27_19\\analyzed\\18-34-45vid

INFO:tensorflow:Restoring parameters from D:\rat movies SC\ar17record\02_27_19\sc_ar17record_02272019-Fassihi-2019-03-05\dlc-models\iteration-1\sc_ar17record_02272019Mar5-trainset95shuffle1\train\snapshot-80000


D:\rat movies SC\ar17record\02_27_19\analyzed\18-27-47video.mp4
Starting %  D:\rat movies SC\ar17record\02_27_19\analyzed\18-27-47video.mp4
Loading  D:\rat movies SC\ar17record\02_27_19\analyzed\18-27-47video.mp4
Duration of video [s]:  1967.0 , recorded with  1.0 fps!
Overall # of frames:  1967 without cropped frame dimensions:  1104 500
Starting to extract posture




  0%|                                                                                         | 0/1967 [00:00<?, ?it/s]

  2%|█▌                                                                              | 38/1967 [00:01<01:14, 25.80it/s]

  3%|██▎                                                                             | 57/1967 [00:02<01:29, 21.29it/s]

  4%|███                                                                             | 76/1967 [00:03<01:39, 19.02it/s]

  5%|███▊                                                                            | 95/1967 [00:05<01:46, 17.55it/s]

  6%|████▌                                                                          | 114/1967 [00:06<01:44, 17.75it/s]

  7%|█████▎                                                                         | 133/1967 [00:07<01:49, 16.75it/s]

  8%|██████                                                                         | 152/1967 [00:08<01:53, 16.06it/s]

  9%|██████▊                  

 66%|███████████████████████████████████████████████████▏                          | 1292/1967 [01:21<00:44, 15.16it/s]

 67%|███████████████████████████████████████████████████▉                          | 1311/1967 [01:23<00:43, 15.05it/s]

 68%|████████████████████████████████████████████████████▋                         | 1330/1967 [01:24<00:40, 15.85it/s]

 69%|█████████████████████████████████████████████████████▍                        | 1349/1967 [01:25<00:39, 15.45it/s]

 70%|██████████████████████████████████████████████████████▏                       | 1368/1967 [01:26<00:39, 15.23it/s]

 71%|███████████████████████████████████████████████████████                       | 1387/1967 [01:28<00:38, 15.07it/s]

 71%|███████████████████████████████████████████████████████▊                      | 1406/1967 [01:29<00:35, 15.89it/s]

 72%|████████████████████████████████████████████████████████▌                     | 1425/1967 [01:30<00:34, 15.51it/s]

 73%|███████████████████████████

...




































































  0%|                                                                                          | 0/874 [00:00<?, ?it/s]


































































  2%|█▊                                                                               | 20/874 [00:01<00:48, 17.61it/s]


































































  3%|██▊                                                                              | 30/874 [00:02<01:15, 11.19it/s]


































































  5%|███▋                                                                             | 40/874 [00:03<01:20, 10.33it/s]


































































  6%|████▋                                                                            | 50/874 [00:05<01:37,  8.44it/s]


































































  7

 51%|█████████████████████████████████████████▏                                      | 450/874 [01:01<01:01,  6.91it/s]


































































 53%|██████████████████████████████████████████                                      | 460/874 [01:02<00:55,  7.42it/s]


































































 54%|███████████████████████████████████████████                                     | 470/874 [01:04<00:58,  6.92it/s]


































































 55%|███████████████████████████████████████████▉                                    | 480/874 [01:05<00:52,  7.44it/s]


































































 56%|████████████████████████████████████████████▊                                   | 490/874 [01:07<00:54,  6.99it/s]


































































 57%|█████████████████████████████████████████████▊                   

Detected frames:  874
Saving results in D:\rat movies SC\ar17record\02_27_19\analyzed...
Saving csv poses!
D:\rat movies SC\ar17record\02_27_19\analyzed\18-54-23video.mp4
Starting %  D:\rat movies SC\ar17record\02_27_19\analyzed\18-54-23video.mp4
Loading  D:\rat movies SC\ar17record\02_27_19\analyzed\18-54-23video.mp4
Duration of video [s]:  374.0 , recorded with  1.0 fps!
Overall # of frames:  374 without cropped frame dimensions:  1104 500
Starting to extract posture






































































  0%|                                                                                          | 0/374 [00:00<?, ?it/s]



































































  5%|████▎                                                                            | 20/374 [00:01<00:20, 17.01it/s]



































































  8%|██████▍                                                                          | 30/374 [00:02<00:31, 11.05it/s]



































































 11%|████████▋                                                                        | 40/374 [00:03<00:32, 10.43it/s]



































































 13%|██████████▊                                                                      | 50/374 [00:05<00:37,  8.54it/s]


































































Detected frames:  374
Saving results in D:\rat movies SC\ar17record\02_27_19\analyzed...
Saving csv poses!
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract any outlier frames!


## Extract outlier frames [optional step]
This is an optional step and is used only when the evaluation results are poor i.e. the labels are incorrectly predicted. In such a case, the user can use the following function to extract frames where the labels are incorrectly predicted. Make sure to provide the correct value of the "iterations" as it will be used to create the unique directory where the extracted frames will be saved.

In [5]:
video=['D:\\rat movies SC\\ar17record\\02_27_19\\analyzed\\18-31-00video.MP4']
path_config_file = 'D:\\rat movies SC\\ar17record\\02_27_19\\sc_ar17record_02272019-Fassihi-2019-03-05\\config.yaml'
deeplabcut.extract_outlier_frames(path_config_file,video,outlieralgorithm='uncertain',p_bound=.6)

network parameters: DeepCut_resnet50_sc_ar17record_02272019Mar5shuffle1_150000
Method  uncertain  found  1294  putative outlier frames.
Do you want to proceed with extracting  20  of those?
If this list is very large, perhaps consider changing the paramters (start, stop, p_bound, comparisonbodyparts) or use a different method.
yes/noyes
Loading video...
Duration of video [s]:  2000.0 , recorded @  1.0 fps!
Overall # of frames:  2000 with (cropped) frame dimensions:  [1104, 500]
Uniformly extracting of frames from 0.0  seconds to 2000.0  seconds.
Let's select frames indices: [562, 483, 1629, 326, 486, 1885, 1547, 332, 1695, 1361, 795, 1897, 1463, 652, 616, 1398, 471, 1786, 1409, 338]
AUTOMATIC ADDING OF VIDEO TO CONFIG FILE FAILED! You need to do this manually for including it in the config.yaml file!
Videopath: D:\rat movies SC\ar17record\02_27_19\analyzed\18-31-00video.MP4 Coordinates for cropping: None
The outlier frames are extracted. They are stored in the subdirectory labeled-data

In [ ]:

path_config_file = 'D:\\rat movies SC\\ar19\\02_07_19\\sc_ar19_02072019-Fassihi4-2019-02-13\\config.yaml' #Enter the path of the config file that was just created from the above step (check the folder)

import os
Mainfolder = 'D:\\rat movies SC\\ar19\\02_07_19'


f = os.listdir(Mainfolder)
text_files = [f for f in os.listdir(Mainfolder) if f.endswith('.avi')]


my_list = text_files
thisApen = Mainfolder+'kir'
thisApen = thisApen.replace("kir", "\\")
string = thisApen
my_new_list = [ string + x for x in my_list]

deeplabcut.extract_outlier_frames(path_config_file,my_new_list)

## Refine Labels [optional step]
Following the extraction of outlier frames, the user can use the following function to move the predicted labels to the correct location. Thus augmenting the training dataset.

In [7]:
%gui wx
deeplabcut.refine_labels(path_config_file)

ERROR:root:Cannot activate multiple GUI eventloops


D:\rat movies SC\ar17record\02_27_19\sc_ar17record_02272019-Fassihi-2019-03-05\labeled-data\18-31-00video
Checking labels if they are outside the image
Closing... The refined labels are stored in a subdirectory under labeled-data. Use the function 'merge_datasets' to augment the training dataset, and then re-train a network using create_training_dataset followed by train_network!


In [8]:
#Once all folders are relabeled, check them and advance. See how to check labels, above!
deeplabcut.merge_datasets(path_config_file)

Merged data sets and updated refinement iteration to 1.
Now you can create a new training set for the expanded annotated images (use create_training_dataset).


## Create a new iteration of training dataset [optional step]
Following the refine labels, append these frames to the original dataset to create a new iteration of training dataset.

In [9]:
deeplabcut.create_training_dataset(path_config_file)

The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


In [ ]:
import os
Mainfolder = 'D:\\Notebook\\sc\\sc-Fassihi-2018-12-16\\videos'
f = os.listdir(Mainfolder)
text_files = [f for f in os.listdir(Mainfolder) if f.endswith('.avi')]


my_list = text_files
thisApen = Mainfolder+'kir'
thisApen = thisApen.replace("kir", "\\")
string = thisApen
my_new_list = [ string + x for x in my_list]
print (my_new_list)



## Create labeled video
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

In [ ]:
deeplabcut.create_labeled_video(path_config_file,my_new_list,save_frames) # my_new_list was created in prevouse cell as all videos in the folder 

## Plot the trajectories of the analyzed videos
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [ ]:
%matplotlib notebook #for making interactive plots.
deeplabcut.plot_trajectories(path_config_file,my_new_list)



In [ ]:
path_config_file = 'D:\\Notebook\\sc\\sc-Fassihi-2018-12-16\\config.yaml'
path_config_file = 'D:\\Notebook\\sc\\sc-Fassihi-2018-12-16\\config.yaml' # Enter the path of the config file that was just created from the above step (check the folder)

videofile_path = 'D:\\Notebook\\sc\\sc-Fassihi-2018-12-16\\videos\\video14-24-01.avi'
#print(path_config_file)
